In [ ]:
%pip install --upgrade --quiet langchain langchain-core langchain-community langchain-openai langchain-groq langchainhub

In [2]:
import os

# os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_API_KEY'
os.environ['GROQ_API_KEY'] = 'YOUR_GROQ_API'

In [8]:
from langchain import hub
from langchain.agents import AgentExecutor, tool
from langchain.agents.output_parsers import XMLAgentOutputParser
# from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

# model = ChatOpenAI()
model = ChatGroq(model_name="mixtral-8x7b-32768")

@tool
def search(query: str) -> str:
    """Pesquise coisas sobre eventos atuais."""
    return "10 degrees"

tool_list = [search]

prompt = hub.pull("hwchase17/xml-agent-convo")

def convert_intermediate_steps(intermediate_steps):
    log = ""
    for action, observation in intermediate_steps:
        log += (
            f"<tool>{action.tool}</tool><tool_input>{action.tool_input}"
            f"</tool_input><observation>{observation}</observation>"
        )
    return log

def convert_tools(tools):
    return "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: convert_intermediate_steps(
            x["intermediate_steps"]
        ),
    }
    | prompt.partial(tools=convert_tools(tool_list))
    | model.bind(stop=["</tool_input>", "</final_answer>"])
    | XMLAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)

In [ ]:
agent_executor.invoke({"input": "Whats the weather in Sao Paulo?"})